In [2]:
!pip install pennylane
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 89.7 MB/s eta 0:00:00


In [11]:
import pennylane as qml
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_iris

# Load Iris dataset
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)

# Normalize data (0 to 1 range)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)

# ---------------------- Quantum Encoding Functions ---------------------- #

# Function to apply Basis Encoding (Fixed Scaling)
def basis_encoding(data, max_bits=4):
    scaled_data = np.round((data - np.min(data)) / (np.max(data) - np.min(data)) * (2**max_bits - 1))
    binary_data = np.concatenate([
        np.array(list(map(int, format(int(x), f'0{max_bits}b')))) for x in scaled_data
    ])

    num_qubits = len(binary_data)
    if num_qubits > 32:
        raise ValueError(f"Too many qubits ({num_qubits}). Reduce feature size.")

    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def circuit():
        for i, bit in enumerate(binary_data):
            if bit == 1:
                qml.PauliX(wires=i)
        return qml.state()

    return circuit()

# Function to apply Amplitude Encoding
def amplitude_encoding(data):
    norm_data = data / np.linalg.norm(data)
    num_qubits = int(np.ceil(np.log2(len(norm_data))))
    padded_data = np.pad(norm_data, (0, 2**num_qubits - len(norm_data)), 'constant')

    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def circuit():
        qml.MottonenStatePreparation(padded_data, wires=list(range(num_qubits)))
        return qml.state()

    return circuit()

# Function to apply Angle Encoding (Optimized)
def angle_encoding(data):
    num_qubits = len(data)

    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def circuit():
        qml.AngleEmbedding(data * np.pi, wires=list(range(num_qubits)), rotation='Y')
        return qml.state()

    return circuit()

# --------------------- Select Data Row & Apply Encoding --------------------- #

row_data = normalized_data[0].astype(float)  # Select first row (no PCA)

# Apply quantum encoding
basis_encoded = basis_encoding(row_data)
amplitude_encoded = amplitude_encoding(row_data)
angle_encoded = angle_encoding(row_data)

# Save encoded data as NumPy arrays
np.save("basis_encoded.npy", basis_encoded)
np.save("amplitude_encoded.npy", amplitude_encoded)
np.save("angle_encoded.npy", angle_encoded)

# Load and verify saved files
loaded_basis = np.load("basis_encoded.npy")
loaded_amplitude = np.load("amplitude_encoded.npy")
loaded_angle = np.load("angle_encoded.npy")

# Display results
print("Normalized Data (First Row):\n", row_data)
print("\nEncoded Data:")
print("Basis Encoded:", loaded_basis)
print("Amplitude Encoded:", loaded_amplitude)
print("Angle Encoded:", loaded_angle)


Normalized Data (First Row):
 [0.22222222 0.625      0.06779661 0.04166667]

Encoded Data:
Basis Encoded: [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
Amplitude Encoded: [0.33262469+0.j 0.93550694+0.j 0.10147872+0.j 0.06236713+0.j]
Angle Encoded: [0.5179962 +0.j 0.03395126+0.j 0.05537332+0.j 0.00362936+0.j
 0.77523609+0.j 0.05081166+0.j 0.08287203+0.j 0.00543172+0.j
 0.1885352 +0.j 0.01235725+0.j 0.02015424+0.j 0.00132098+0.j
 0.28216286+0.j 0.01849393+0.j 0.03016295+0.j 0.00197698+0.j]
